<a href="https://colab.research.google.com/github/johaewon/KBSC/blob/main/KcELECTRA_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install soynlp emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 416 kB 30.1 MB/s 
     |████████████████████████████████| 197 kB 65.1 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=b97e831f64f8fafc02fd6ca8507ac43f8584df6f99573a687963dc7879bde80f
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 24.0 MB/s 
     |████████████████████████████████| 6.6 MB 53.1 MB/s 
     |████████████████████████████████| 120 kB 66.6 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, AdamW 
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
device = torch.device("cuda")

In [ ]:
test_data = pd.read_excel('/content/drive/MyDrive/한국어_단발성_대화_데이터셋.xlsx')

In [ ]:
test_data[:10]

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,공포,5468
0,언니 동생으로 부르는게 맞는 일인가요..??,공포,NaN,NaN,NaN,놀람,5898.0
1,그냥 내 느낌일뿐겠지?,공포,NaN,NaN,NaN,분노,5665.0
2,아직너무초기라서 그런거죠?,공포,NaN,NaN,NaN,슬픔,5267.0
3,유치원버스 사고 낫다던데,공포,NaN,NaN,NaN,중립,4830.0
4,근데 원래이런거맞나요,공포,NaN,NaN,NaN,행복,6037.0
5,남자친구가 떠날까봐요,공포,NaN,NaN,NaN,혐오,5429.0
6,이거 했는데 허리가 아플수도 있나요? ;;,공포,NaN,NaN,NaN,Total,38594.0
7,내가불안해서꾸는걸까..,공포,NaN,NaN,NaN,NaN,NaN
8,일주일도 안 남았당...ㅠㅠ,공포,NaN,NaN,NaN,NaN,NaN
9,약은 최대한 안먹으려고 하는데좋은 음시있나요?0,공포,NaN,NaN,NaN,NaN,NaN


In [ ]:
test_data.loc[(test_data['Emotion'] == "공포"), 'Emotion'] = 0 
test_data.loc[(test_data['Emotion'] == "놀람"), 'Emotion'] = 1  
test_data.loc[(test_data['Emotion'] == "분노"), 'Emotion'] = 2  
test_data.loc[(test_data['Emotion'] == "슬픔"), 'Emotion'] = 3  
test_data.loc[(test_data['Emotion'] == "중립"), 'Emotion'] = 4  
test_data.loc[(test_data['Emotion'] == "행복"), 'Emotion'] = 5  
test_data.loc[(test_data['Emotion'] == "혐오"), 'Emotion'] = 6

In [ ]:
train_dataset = []
for sen, label in zip(test_data['Sentence'], test_data['Emotion']):
  data_train = []
  data_train.append(sen)
  data_train.append(str(label))

  train_dataset.append(data_train)

In [ ]:
print(len(test_data))

38594


In [ ]:
class TrainDataset(Dataset):
  
  def __init__(self, dataset):
    self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")

    self.sentences = [str([i[0]]) for i in dataset]
    self.labels = [np.int32(i[1]) for i in dataset]

  def __len__(self):
    return (len(self.labels))
  
  def __getitem__(self, i):
    text = self.sentences[i]
    y = self.labels[i]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [ ]:
train_dataset = TrainDataset(train_dataset)

In [ ]:
print(train_dataset[0])

(tensor([    2,    61,    11, 24294, 13322,  7973, 17986,  4131,  8660, 21047,
         4071,    18,    18,    33,    33,    11,    63,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 0)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2329: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
from torch import nn

model = AutoModel.from_pretrained("beomi/KcELECTRA-base", num_labels=363)
# model.classifier = torch.nn.Sequential(
#                                         nn.Linear(768, 768, bias=True),
#                                         nn.Dropout(p=0.1, inplace=False),
#                                         nn.Linear(768, 7, bias=True))
model = model.to(device)

# 한번 실행해보기
# text, attention_mask, y = train_dataset[0]
# model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model

ElectraModel(
  (embeddings): ElectraEmbeddings(
    (word_embeddings): Embedding(50135, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): ElectraEncoder(
    (layer): ModuleList(
      (0): ElectraLayer(
        (attention): ElectraAttention(
          (self): ElectraSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): ElectraSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inpl

In [ ]:
batch_size = 32
epochs = 2

In [ ]:
optimizer = AdamW(model.parameters(), lr=3e-5)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=5, shuffle=True)

In [ ]:
losses = []
accuracies = []

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

loss_fn = nn.CrossEntropyLoss()

for i in range(epochs):
  train_acc = 0.0
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_dataloader):
    optimizer.zero_grad()
    y_batch = y_batch.long().to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    y_pred = y_pred[:, -1, :]
    loss = loss_fn(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    train_acc += calc_accuracy(y_pred, y_batch)
    total += len(y_batch)

    batches += 1
    if batches % 50 == 0:
      print("epoch {} loss {} train acc {}".format(i+1, loss.data.cpu().numpy(), train_acc / (batches+1)))
  print("epoch {} loss {} train acc {}".format(i+1, loss.data.cpu().numpy(), train_acc / (batches+1)))
  model.eval()

  0%|          | 0/1207 [00:00<?, ?it/s]

epoch 1 loss 3.2658233642578125 train acc 0.31862745098039214
epoch 1 loss 3.456354856491089 train acc 0.3527227722772277
epoch 1 loss 2.657909631729126 train acc 0.38141556291390727
epoch 1 loss 3.206397771835327 train acc 0.3933457711442786
epoch 1 loss 2.2712299823760986 train acc 0.4086155378486056
epoch 1 loss 3.0356028079986572 train acc 0.4214078073089701
epoch 1 loss 2.0910816192626953 train acc 0.43189102564102566
epoch 1 loss 2.1174867153167725 train acc 0.4393703241895262
epoch 1 loss 1.9977515935897827 train acc 0.44595343680709537
epoch 1 loss 2.5776002407073975 train acc 0.4537175648702595
epoch 1 loss 2.939138650894165 train acc 0.4587114337568058
epoch 1 loss 3.020443916320801 train acc 0.4636023294509151
epoch 1 loss 2.4805097579956055 train acc 0.4662058371735791
epoch 1 loss 1.9846450090408325 train acc 0.46995363766048504
epoch 1 loss 2.4906351566314697 train acc 0.474034620505992
epoch 1 loss 2.37032151222229 train acc 0.4766697877652934
epoch 1 loss 2.661782741546

  0%|          | 0/1207 [00:00<?, ?it/s]

epoch 2 loss 2.4416277408599854 train acc 0.5778186274509803
epoch 2 loss 1.8112006187438965 train acc 0.5801361386138614
epoch 2 loss 1.3743337392807007 train acc 0.5807119205298014
epoch 2 loss 1.857951283454895 train acc 0.5883084577114428
epoch 2 loss 1.349480152130127 train acc 0.5902639442231076
epoch 2 loss 1.8771840333938599 train acc 0.5871054817275747
epoch 2 loss 2.7986834049224854 train acc 0.5896545584045584
epoch 2 loss 1.899599552154541 train acc 0.5925810473815462
epoch 2 loss 1.933864712715149 train acc 0.593680709534368
epoch 2 loss 1.979242205619812 train acc 0.593438123752495
epoch 2 loss 1.822255253791809 train acc 0.5932395644283122
epoch 2 loss 1.6977189779281616 train acc 0.5928660565723793
epoch 2 loss 1.3738789558410645 train acc 0.5947100614439325
epoch 2 loss 2.083326578140259 train acc 0.5943741084165478
epoch 2 loss 2.295665979385376 train acc 0.5955808921438083
epoch 2 loss 1.4806292057037354 train acc 0.5955056179775281
epoch 2 loss 1.6281639337539673 tr

In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/Colab Notebooks/model.pt")

In [ ]:
def predict(sentence):
    data = [sentence, '0']
    dataset_another = [data]
    logits = 0
    another_test = TrainDataset(dataset_another)
    test_dataloader = torch.utils.data.DataLoader(another_test)

    model.eval()

    for input_ids_batch, attention_masks_batch, y_batch in test_dataloader:
        y_batch = y_batch.long().to(device)
        out = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
        out = out[:, -1, :]

        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()
            logits = np.argmax(logits)
    return logits

In [ ]:
predict("어젯밤에 집에 왔다. 집에 오는 길에 마늘빵을 사 와서 먹었는데 너무 맛있어서 행복했다. 근데 앱 개발이 제대로 안 돼서 개빡쳤음. 마늘빵이고 뭐고 다 던져버려야지...")

6